In [77]:
import os
import math
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)

for x, folder, file in os.walk(datasheets):
    
    try: 
        
        for loc in folder:

            print(loc)

            folder_loc = datasheets + loc

            glob.os.chdir(folder_loc)

            all_datasheets = glob.glob(folder_loc + "/*.csv")

            N_Re_Run = []
            U_Re_Run = []

            for sheet in all_datasheets:

                print(sheet)

                lab_data = pd.read_csv(sheet)

                sheet_name = sheet.split('\\')[-1]
                title = sheet_name.split('_')[2]
            # Checking for data - skipping remainder of code (breaking loop) if there is no data

                if np.isnan(lab_data.Absorbance[0]):
                    break

                    # Continuing if data is found

                else:
                    print(sheet)


                        # Identifying standard absorbances based on 'Type' column, and adding to list of values to plot

                    curve_absorbance = []
                    curve_concentration = []

                    for n in range(len(lab_data)):
                        if lab_data.Type[n] == 'C':
                            curve_absorbance.append(float(lab_data.Absorbance[n]))
                            curve_concentration.append(float(lab_data.Sample_ID[n]))


                    # calculating r^2 value for two lists of values

                slope, intercept, r_value, p_value, std_err = stats.linregress(curve_absorbance, curve_concentration)

                    # creating conditional for screening based on r^2 value
                if r_value**2 < 0.8:

                        # Moving bad curve sheet to new folder to avoid running again in the future
                    shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Bad_Curves/BADCURVE_' + str(sheet_name))

                        # Adding sample id information to list of re-runs if curve was bad

                    for i, row in lab_data.iterrows():
                        if row['Type'] == 'O':
                            if title == "URE":
                                U_Re_Run.append(row['Sample_ID'])
                            elif title == "NIT":
                                N_Re_Run.append(row['Sample_ID'])
                        elif row['Type'] == 'D':
                            if title == "URE":
                                U_Re_Run.append(row['Sample_ID'])
                            elif title == "NIT":
                                N_Re_Run.append(row['Sample_ID'])


                else: 
                        # define which rows contain data for samples, and creating new df with only absorbance

                    sample_data = lab_data[lab_data.Type != 'C']
                    sample_data = sample_data[sample_data.Type != 'B']
                    sample_data_ab = sample_data[['Sample_ID', 'Absorbance']]


                        # group sample data based on Sample_ID information

                    sample_data_g = sample_data_ab.groupby('Sample_ID')


                        # merge and create new dataframe for variance values

                    sample_data_var = sample_data_ab.merge((abs((sample_data_g.max() - sample_data_g.min())) / (sample_data_g.max())), on='Sample_ID')


                        # create conditional based on variance column

                    for i, row in sample_data_var.iterrows():
                        if float(row['Absorbance_y']) > 0.1:

                                # Adding sample id information to list if condition is met
                            if title == "URE":
                                U_Re_Run.append(row['Sample_ID'])
                            elif title == "NIT":
                                N_Re_Run.append(row['Sample_ID'])

                    shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Lab_Sheets/' + loc + '/Completed/' + str(sheet_name))

                # Adding list of ID's to fill column in dataframe
            set_ = sheet_name.split('_')[1]
            title1 = sheet_name.split('_')[3]
            title2 = sheet_name.split('_')[4]
            titles = (title1, title2)
            file_name = ('_').join(titles)
            file_name  

                # Creating a new dataframe with sample ID's to be run again

            if U_Re_Run != []:

                samp_set = int(input('You have ' + str(len(U_Re_Run)/2) + ' unique samples (not including duplicates) to re-run for Ureides in ' + str(loc) + '. How many of these do you wish to run per set?'))
                print("Creating " + str(math.ceil(len(U_Re_Run)/(samp_set*2))) + " sheets for " + str(len(U_Re_Run)) + " total samples.")

                U_Re_Run = pd.DataFrame(U_Re_Run)

                counter = 0

                    # Creating sheets based on number of samples to be displayed on each sheet (set size)

                while counter <= math.ceil(len(U_Re_Run)/(samp_set*2)):

                    for i in range(math.ceil(len(U_Re_Run)/(samp_set*2))):

                        template2 = U_Re_Run[(i*samp_set*2):((1+i)*samp_set*2)]


                                # Inserting rows for blanks and curve data in nitrates through a list

                        utop = []

                        utop.insert(0, 4396.660764)
                        utop.insert(0, 4396.660764)
                        utop.insert(0, 4396.660764)
                        utop.insert(0, 1099.165191)
                        utop.insert(0, 1099.165191)
                        utop.insert(0, 1099.165191)
                        utop.insert(0, 549.5825955)
                        utop.insert(0, 549.5825955)
                        utop.insert(0, 549.5825955)
                        utop.insert(0, 0.00)
                        utop.insert(0, 0.00)
                        utop.insert(0, 0.00)
                        utop.insert(0, '2_2')
                        utop.insert(0, '2_1')
                        utop.insert(0, '1_2')
                        utop.insert(0, '1_1')


                        template2 = pd.concat([pd.DataFrame(utop), template2], ignore_index = True)


                                # Creating values for sample type

                        seq = cycle(['O', 'D'])

                        type_list = ['B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']


                                # Creating new columns in dataframe and appending corresponding data 
                                # ... or leaving rows empty for lab to fill in later

                        template2['Type'] = [next(seq) for i in range(len(template2))]
                        template2.Type[0:16] = type_list
                        template2['Sample_Wt(g)'] = ''
                        template2['Absorbance'] = ''
                        template2 = template2[['Type', 'Sample_Wt(g)', 0, 'Absorbance']]
                        template2.columns = ['Type', 'Sample_Wt(g)', 'Sample_ID', 'Absorbance']

                        glob.os.chdir(datasheets + '/' + str(loc))

                                # Saving unique nitrate and ureide datasets in new location with unique names
                                # based on set# ... date is blank for user to change later when sampes are run

                        if sheet_name.split('_')[0] == 'A':
                            template2.to_csv('B_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as B_xx_URE_' + file_name + '_00_00_00.csv')
                        elif sheet_name.split('_')[0] == 'B':
                            template2.to_csv('C_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as C_xx_URE_' + file_name + '_00_00_00.csv')
                        elif sheet_name.split('_')[0] == 'C':
                            template2.to_csv('D_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as D_xx_URE_' + file_name + '_00_00_00.csv')
                        elif sheet_name.split('_')[0] == 'D':
                            template2.to_csv('E_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as E_xx_URE_' + file_name + '_00_00_00.csv')
                        else:
                            template2.to_csv('xx_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as xx_xx_URE_' + file_name + '_00_00_00.csv')

                        counter = counter + 2

            if N_Re_Run != []:
                samp_set = int(input('You have ' + str(len(N_Re_Run)/2) + ' unique samples (not including duplicates) to re-run for Nitrates in ' + str(loc) + '. How many of these do you wish to run per set?'))
                print("Creating " + str(math.ceil(len(N_Re_Run)/(samp_set*2))) + " sheets for " + str(len(N_Re_Run)) + " total samples.")

                N_Re_Run = pd.DataFrame(N_Re_Run)

                counter = 0

                    # Creating sheets based on number of samples to be displayed on each sheet (set size)

                while counter <= math.ceil(len(N_Re_Run)/(samp_set*2)):

                    for i in range(math.ceil(len(N_Re_Run)/(samp_set*2))):

                        template2 = N_Re_Run[(i*samp_set*2):((1+i)*samp_set*2)]


                                # Inserting rows for blanks and curve data in nitrates through a list

                        ntop = []

                        ntop.insert(0, 4500.00)
                        ntop.insert(0, 4500.00)
                        ntop.insert(0, 4500.00)
                        ntop.insert(0, 3000.00)
                        ntop.insert(0, 3000.00)
                        ntop.insert(0, 3000.00)
                        ntop.insert(0, 1500.00)
                        ntop.insert(0, 1500.00)
                        ntop.insert(0, 1500.00)
                        ntop.insert(0, 0.00)
                        ntop.insert(0, 0.00)
                        ntop.insert(0, 0.00)
                        ntop.insert(0, '2_2')
                        ntop.insert(0, '2_1')
                        ntop.insert(0, '1_2')
                        ntop.insert(0, '1_1')

                        template2 = pd.concat([pd.DataFrame(ntop), template2], ignore_index = True)


                                # Creating values for sample type

                        seq = cycle(['O', 'D'])

                        type_list = ['B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']


                                # Creating new columns in dataframe and appending corresponding data 
                                # ... or leaving rows empty for lab to fill in later

                        template2['Type'] = [next(seq) for i in range(len(template2))]
                        template2.Type[0:16] = type_list
                        template2['Sample_Wt(g)'] = ''
                        template2['Absorbance'] = ''
                        template2 = template2[['Type', 'Sample_Wt(g)', 0, 'Absorbance']]
                        template2.columns = ['Type', 'Sample_Wt(g)', 'Sample_ID', 'Absorbance']

                        glob.os.chdir(datasheets + '/' + str(loc))

                                # Saving unique nitrate and ureide datasets in new location with unique names
                                # based on set# ... date is blank for user to change later when sampes are run

                        if sheet_name.split('_')[0] == 'A':
                            template2.to_csv('B_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as B_xx_NIT_' + file_name + '_00_00_00.csv')
                        elif sheet_name.split('_')[0] == 'B':
                            template2.to_csv('C_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as C_xx_NIT_' + file_name + '_00_00_00.csv')
                        elif sheet_name.split('_')[0] == 'C':
                            template2.to_csv('D_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as D_xx_NIT_' + file_name + '_00_00_00.csv')
                        elif sheet_name.split('_')[0] == 'D':
                            template2.to_csv('E_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as E_xx_NIT_' + file_name + '_00_00_00.csv')
                        else:
                            template2.to_csv('xx_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            print('New lab sheets saved as xx_xx_NIT_' + file_name + '_00_00_00.csv')

                        counter = counter + 2

    except FileNotFoundError:
        continue

MR_USB
RV_CTS
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/RV_CTS\A_1_URE_RV_CTS_00_00_00.csv
USB_Indiana
USB_Iowa
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/USB_Iowa\G_1_NIT_LMR_IA_00_00_00.csv
USB_Minnesota
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/USB_Minnesota\D_1_NIT_LMR_MN_00_00_00.csv
USB_South_Dakota
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/USB_South_Dakota\A_1_NIT_LMR_SD_USB_00_00_00.csv
Completed
Completed
Completed
Completed
Completed
Completed
